# Spring's constant measuring

TODO:
analyze dinamic dataset
create the plots
get the coefficients with linear regression
use chi2 to see if its correct

## Importing libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import scipy.signal as spy
import math

from typing import Dict, List

## Loading data

### reading filenames

In [ ]:
data_dir = "./data"

csv_data = []
excel_data = []

for root, dirs, files in os.walk(data_dir):
    for file in files:
        current_file_path = os.path.join(root, file).replace(
            "\\", "/"
        )  # fix the unbearably frustrating flaws of the wanna-be OS... "Windows"
        if "xlsx" in file:
            excel_data.append(current_file_path)
        elif "csv" in file:
            csv_data.append(current_file_path)
    break

del root, dirs, files, current_file_path, data_dir

### functions to read the files

In [ ]:
def read_csv(filename: str):
    data = pd.read_csv(filename, sep=";").replace(",", ".", regex=True)

    data.dropna(inplace=True)
    data.drop(
        index=data.index[0], axis=0, inplace=True
    )  # instrument error causes first value to be nonsensical

    for col in data.columns:
        data[col] = data[col].apply(pd.to_numeric)

    return data


def read_excel(filename: str):
    data = pd.read_excel(filename)
    return data

### Load the datasets to a dict

In [ ]:
datasets = dict()

for file in csv_data + excel_data:
    key = os.path.basename(file).split(".")[0]
    if "csv" in file:
        datasets[key] = read_csv(file)
    elif "xlsx" in file:
        datasets[key] = read_excel(file)

for k in datasets.keys():
    print(f"\nkey: {k}")
    datasets[k].info()

del k, key, file, csv_data, excel_data

In [ ]:
# sampling frequencies in Hz

static_sampling_frequency = 100
dynamic_sampling_frequency = 200

In [ ]:
static_keys = list(filter(lambda k: "statico" in k, datasets.keys()))
dynamic_keys = list(filter(lambda k: "dinamico" in k, datasets.keys()))


static_datasets = {k: datasets[k] for k in static_keys}
dynamic_datasets = {k: datasets[k] for k in dynamic_keys}

del static_keys, dynamic_keys

In [ ]:
object_masses = dict()
ds = datasets["misure_masse"]
objects = [str(e) for e in ds.iloc[:, 0].array.tolist()]
masses = ds.iloc[:, 1].array.tolist()

for i in range(len(objects)):
    object_masses[objects[i]] = masses[i]
print(object_masses)

del ds, objects, masses, i, datasets

In [ ]:
def calculate_total_mass(
    used_masses: str, spring_used: str, has_support_structure: bool = True
):
    used_masses = str(used_masses)
    total_mass = object_masses[spring_used]
    if has_support_structure:
        total_mass += object_masses["piatto"] + object_masses["supporto"]

    for m in [s for s in used_masses.split(",") if s]:
        if m in object_masses.keys():
            total_mass += object_masses[m]

    return total_mass

## Analyze data

### Functions for linear regression and chi2

$$ y = Ax + B$$

In [ ]:
def LinRef(*, x: List[float], y: List[float]) -> Dict[str, float]:
    pass


def chi2(
    *, Ei: List[float], Oi: List[float], sigmai: List[float], deg_freedom: int
) -> Dict[str, float]:
    pass

### Analyze static

```results``` nella forma di: [[massa * 9.805 (quindi il peso), lunghezza], ...]

In [ ]:
def analyze_calibre_data(dataset_name: str):
    used_spring = "molla " + dataset_name[8:-8:].replace("-", " ")

    used_masses = static_datasets[dataset_name].iloc[:, 0].array.tolist()
    mm_lengths = static_datasets[dataset_name].iloc[:, 1].array.tolist()
    # in_lengths = static_datasets[dataset_name].iloc[:, 2].array.tolist()

    print(f"\nDataset: {dataset_name}\nSpring: {used_spring}\nMasses: {used_masses}")

    results = np.ndarray((len(used_masses), 2))

    for idx, used_masses in enumerate(used_masses):
        results[idx] = [
            9.805 * calculate_total_mass(used_masses, used_spring),
            mm_lengths[idx],
        ]

    print("Results in array form (weight-length):", results, end="\n------------\n")
    return results


def analyze_sonar_data(dataset_name: str):
    used_spring = "molla " + dataset_name[6:-8:].replace("-", " ")

    used_masses = static_datasets["masse_" + dataset_name].iloc[:, 0].array.tolist()
    runs = static_datasets[dataset_name]

    print(f"\nDataset: {dataset_name}\nSpring: {used_spring}\nMasses: {used_masses}")

    results = np.ndarray((len(used_masses), 2))

    for idx, run_name in enumerate(runs.columns):
        # print(runs[run_name].array)
        signal = runs[run_name]
        results[idx] = [
            9.805 * calculate_total_mass(used_masses[idx], used_spring),
            np.mean(signal),
        ]

    print("Results in array form (weight-length):\n", results, end="\n------------\n")
    return results


def analyze_static_dataset(dataset_name: str):
    if "calibro" in dataset_name:
        return analyze_calibre_data(dataset_name)
    elif "sonar" in dataset_name:
        return analyze_sonar_data(dataset_name)


final_static_data = dict()

for key in filter(lambda e: not "masse" in e, static_datasets.keys()):
    final_static_data[key] = analyze_static_dataset(key)

del key

### Analyze dinamic

In [ ]:
def calculate_mean_period(signal: list):
    peaks, _ = spy.find_peaks(signal, width=10)
    mean = np.mean(signal)
    peaks = list(filter(lambda e: e >= mean, peaks))

    acc_mean = 0
    for i in range(len(peaks) - 1):
        acc_mean += abs(peaks[i] - peaks[i + 1]) / dynamic_sampling_frequency
    mean = acc_mean / (len(peaks) - 1)

    acc_sigma = 0
    for i in range(len(peaks) - 1):
        acc_sigma += (
            abs((peaks[i] - peaks[i + 1]) / dynamic_sampling_frequency) - mean
        ) ** 2
    sigma = math.sqrt(acc_sigma / (len(peaks) - 1))

    return (mean, sigma / math.sqrt(len(peaks) - 1))

Graph of the oscillations

In [ ]:
"""
plt.figure(figsize=(16, 9))
# test run:
for j in range(len(dynamic_datasets["sonar_non-pretensionata_dinamico"])):
    _signal = (
        dynamic_datasets["sonar_non-pretensionata_dinamico"].iloc[:, j].array.tolist()
    )
    _peaks, _ = spy.find_peaks(_signal, width=10)
    _mean = np.mean(_signal)
    _peaks = list(filter(lambda e: e >= _mean, _peaks))

    print(len(_peaks))

    plt.title("Molla non pretensionata dinamico")
    plt.xlabel("Tempo [s]")
    plt.ylabel("Spostamento [mm]")
    plt.plot(
        [i / dynamic_sampling_frequency for i in range(len(_signal))], _signal, lw=1.2
    )
    plt.scatter(
        [x / dynamic_sampling_frequency for x in _peaks],
        [_signal[x] for x in _peaks],
        marker="x",
        s=21,
    )

    _acc = 0
    for i in range(len(_peaks) - 1):
        _acc += abs(_peaks[i] - _peaks[i + 1]) / dynamic_sampling_frequency
    _mean = _acc / (len(_peaks) - 1)

    print(_mean, _mean / math.sqrt(len(_peaks) - 1))

del _signal
del _peaks
del _mean
del _acc
del _
"""

### Show values

### Make graphs